# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [2]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 0.8230,  0.4084, -1.7022,  ...,  0.5817, -0.4647,  0.0430],
        [ 0.3129, -0.1957,  1.0494,  ...,  2.1296, -0.4952, -0.4005],
        [ 0.0298,  0.4907, -1.6068,  ...,  3.1417, -0.2157,  0.6175],
        ...,
        [ 0.0251, -0.2263,  0.0561,  ..., -3.1176,  0.2435, -1.1129],
        [-0.0128,  3.6077,  0.0401,  ..., -0.0161,  0.5667, -0.0275],
        [ 0.2217,  0.1113,  0.1529,  ..., -0.2236,  0.2655, -0.0615]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [3]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-1.4067e-01,  7.4505e-03, -4.9658e-01,  ...,  1.4874e-01,
         -1.1732e-01, -1.7631e+00],
        [-4.8641e-01, -2.4344e-01, -6.9215e-01,  ...,  1.1420e+00,
          2.1655e-01, -7.7601e-01],
        [ 4.3029e-01,  2.0397e-01,  6.4740e-02,  ..., -9.8337e-02,
          3.5841e-01,  3.6059e+00],
        ...,
        [-1.0339e-03,  1.4886e-01, -2.5264e-01,  ...,  6.7513e-01,
         -7.9912e-02, -2.7097e-01],
        [-2.1062e-01,  8.4287e-01,  1.2489e-01,  ...,  3.8972e-01,
         -4.4127e-01, -1.7925e+00],
        [ 4.4976e-01,  5.3314e+00, -1.5828e+00,  ...,  7.5773e-02,
         -3.5293e-03,  1.4975e-01]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-3.1762e-01, -8.4908e-02,  1.3137e+00,  ...,  1.7080e-01,
         -5.8452e-01,  4.8756e-01],
        [ 4.5867e-01,  2.0717e+00,  6.4167e-01,  ...,  6.7605e-02,
         -2.1626e-01, -1.0260e+00],
        [-5.9631e-01, -2.0202e-01, -1.5413e+00,  ..., -4.2742e-01,
          4.8864e-01, -2.0776e-01],
        ...,
        [ 7.7665e-02, -2.0013e-01,  1.2192e+00,  ..., -8.8213e-01,
          6.7617e-01,  4.3024e-01],
        [-3.1765e-01,  8.6555e-01,  1.7981e+00,  ..., -3.9858e-01,
          3.5127e-01, -2.4854e-01],
        [ 2.2397e-03,  1.9636e-01, -1.6901e+00,  ..., -9.8352e-01,
         -5.0752e-02,  2.8529e+00]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

GPUs have far more processor cores than CPUs, but because each GPU core runs significantly slower than a CPU core and do not have the features needed for modern operating systems, they are not appropriate for performing most of the processing in everyday computing. They are most suited to compute-intensive operations like training NN.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

The cost to build GPUs are way more expensive than CPUs.